In [1]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import ShuffleSplit

names=['Type','LifeStyle','Vacation','eCredit','salary','property','label']
train_data=pd.read_csv("training.txt",header=None,names=names)
print train_data.head()

      Type     LifeStyle  Vacation  eCredit  salary  property label
0  student  spend>saving         6       40   13.62    3.2804    C1
1  student  spend>saving        11       21   15.32    2.0232    C1
2  student  spend>saving         7       64   16.55    3.1202    C1
3  student  spend>saving         3       47   15.71    3.4022    C1
4  student  spend>saving        15       10   16.96    2.2825    C1


/home/abdul/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def class_counts(rows):
    """Counts the number of each type of example in a dataset."""
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        # in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [3]:
train_data=train_data.values.tolist()
class_counts(train_data)

{'C1': 36, 'C2': 26, 'C3': 41, 'C4': 47, 'C5': 36}

In [4]:
def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

In [40]:
class Question:
    """A Question is used to partition a dataset.

    This class just records a 'column number' (e.g., 0 for Color) and a
    'column value' (e.g., Green). The 'match' method is used to compare
    the feature value in an example to the feature value stored in the
    question. See the demo below.
    """

    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]

        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            names[self.column], condition, str(self.value))

In [41]:
def partition(rows, question):
    """Partitions a dataset.

    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [42]:
true_rows, false_rows = partition(train_data, Question(2, 17))

In [43]:
print true_rows[:3]

[['student', 'spend>>saving', 22, 38, 13.88, 0.7394, 'C1'], ['student', 'spend>>saving', 25, 30, 15.64, 3.1282, 'C1'], ['student', 'spend>>saving', 31, 15, 14.4, 2.3925, 'C1']]


In [44]:
def gini(rows):
    """Calculate the Gini Impurity for a list of rows.

    There are a few different ways to do this, I thought this one was
    the most concise. See:
    https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    """
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [45]:
def info_gain(left, right, current_uncertainty):
    """Information Gain.

    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [46]:
current_uncertainty = gini(train_data)
current_uncertainty

0.793097467915366

In [47]:
true_rows, false_rows = partition(train_data, Question(2, 17))
info_gain(true_rows, false_rows, current_uncertainty)

0.16835776234541489

In [48]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val)

            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [49]:
find_best_split(train_data)

(0.16835776234541489, Is Vacation >= 17?)

In [50]:
class Leaf:
    """A Leaf node classifies data.

    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """

    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [51]:
class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [52]:
def build_tree(rows):
    """Builds the tree.

    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """

    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_split(rows)

    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)

In [53]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [54]:
my_tree = build_tree(train_data)

In [55]:
print_tree(my_tree)

Is Vacation >= 17?
--> True:
  Is eCredit >= 24?
  --> True:
    Is property >= 5.7642?
    --> True:
('      Predict', {'C1': 6})
    --> False:
      Is salary >= 18.23?
      --> True:
        Is Type == engineer?
        --> True:
          Is eCredit >= 38?
          --> True:
            Is Vacation >= 26?
            --> True:
              Is eCredit >= 82?
              --> True:
('                Predict', {'C1': 2})
              --> False:
                Is eCredit >= 62?
                --> True:
('                  Predict', {'C4': 7})
                --> False:
                  Is salary >= 20.08?
                  --> True:
                    Is salary >= 20.82?
                    --> True:
                      Is property >= 3.8187?
                      --> True:
('                        Predict', {'C4': 2})
                      --> False:
                        Is salary >= 20.91?
                        --> True:
('                          Predict', {'C1': 

In [56]:
def classify(row, node):
    """See the 'rules of recursion' above."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [57]:
classify(train_data[0], my_tree)

{'C1': 10}

In [58]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [59]:
print_leaf(classify(train_data[0], my_tree))

{'C1': '100%'}

In [60]:
print_leaf(classify(train_data[100], my_tree))

{'C3': '100%'}

In [61]:
test_data=pd.read_csv("testing.txt",header=None,names=names)
test_data=test_data.values.tolist()
for row in test_data:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))

Actual: C1. Predicted: {'C1': '100%'}
Actual: C1. Predicted: {'C1': '100%'}
Actual: C1. Predicted: {'C1': '100%'}
Actual: C1. Predicted: {'C2': '100%'}
Actual: C1. Predicted: {'C2': '100%'}
Actual: C1. Predicted: {'C2': '100%'}
Actual: C1. Predicted: {'C2': '100%'}
Actual: C1. Predicted: {'C2': '100%'}
Actual: C1. Predicted: {'C3': '100%'}
Actual: C1. Predicted: {'C3': '100%'}
Actual: C1. Predicted: {'C3': '100%'}
Actual: C1. Predicted: {'C3': '100%'}
Actual: C1. Predicted: {'C5': '100%'}
Actual: C1. Predicted: {'C4': '100%'}
Actual: C1. Predicted: {'C4': '100%'}
Actual: C1. Predicted: {'C4': '100%'}
Actual: C1. Predicted: {'C4': '100%'}
Actual: C1. Predicted: {'C4': '100%'}
Actual: C1. Predicted: {'C5': '100%'}
Actual: C1. Predicted: {'C5': '100%'}
Actual: C1. Predicted: {'C5': '100%'}
